In [1]:
import pandas as pd
import networkx
from tqdm import tqdm
import pickle
import itertools


## Organizing pandas

### 1. Coordinates (Node information file)

In [ ]:
coordinates = pd.read_csv(r"data\USA-road-d.CAL.co", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1')

In [ ]:
# combining two columns pandas
coordinates['combined']=coordinates['c'].astype(str)+''+coordinates['9th'].astype(str)

In [ ]:
coordinates = coordinates.iloc[6:] # deleting the first 5 rows

In [ ]:
# organizing pandas DF
coordinates.drop(['c', '9th', 'Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

In [ ]:
coordinates.columns = ['Latitude', 'Longitude', 'Id_Node']

In [ ]:
coordinates = coordinates.reset_index(drop=True)

In [ ]:
# Transform all str into int
coordinates[['Latitude', 'Longitude']] = coordinates[['Latitude', 'Longitude']].astype(int)

In [ ]:
coordinates.head(20)

### 2. Distance graph

In [ ]:
distance = pd.read_csv(r"data\USA-road-d.CAL.gr", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1', error_bad_lines=False)

In [ ]:
distance = distance.iloc[6:] # deleting the first 5 rows

In [ ]:
# organizing pandas DF
distance.drop(['c','Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

(Id_Node1, Id_Node2, d(Id_Node1,Id_Node2)), where d(x,y) is the physical distance between x and y.

In [ ]:
distance.columns = ['Id_Node1', 'Id_Node2', 'Distance']

In [ ]:
distance = distance.reset_index(drop=True)

In [ ]:
# Transform all str into int
distance = distance.astype(int)

In [ ]:
distance.head(20)

### 3. Time travel graph

In [ ]:
travel = pd.read_csv(r"data\USA-road-t.CAL.gr", sep=' ', delimiter=None,
                      index_col=None, usecols=None, encoding='ISO-8859-1', error_bad_lines = False)

In [ ]:
travel = travel.iloc[6:] # deleting the first 5 rows

In [ ]:
# organizing pandas DF
travel.drop(['c','Challenge:', 'Shortest', 'Paths'], axis=1, inplace=True)

(Node1, Node2, t(Id_Node1, Id_Node2))

In [ ]:
travel.columns = ['Id_Node1', 'Id_Node2', 'Time distance']

In [ ]:
travel = travel.reset_index(drop=True)

In [ ]:
travel.head(20)

In [ ]:
# Transform all str into int
travel = travel.astype(int)

## Part 1. Create data types for nodes and edges

In [ ]:
# Create a list of all nodes
nodes =  []
nodes.append([i for i in range (1,1890816)])

In [ ]:
# Create a list of set with (Node_1, Node_2)
edges = []

In [ ]:
temp_travel = travel.drop(['Time distance'], axis=1)
for index, element in tqdm(temp_travel.iterrows()):
    edge = [] # couples of edges
    edge.extend([element[0], element[1]])
    edges.append(edge)

In [ ]:
edges

We have to find out if all verteces are bidirectional, to state that it's a bidirectional or directional graph. <br/>
Since the data downloaded talkes about Arcs instead of Edges we can assume that it's a directed graph

In [ ]:
with open('outfile', 'wb') as fp:
    pickle.dump(edges, fp)

In [62]:
with open ('outfile', 'rb') as fp:
    edges = pickle.load(fp)

### Create the adjacency dictionary

In [ ]:
adj_dic = {}

In [ ]:
for i in tqdm(edges):
    if i[0] not in adj_dic: # check if first node is a key already 
        
        adj_dic[i[0]] = []
        adj_dic[i[0]].append(i[1])
    
    else:
        
        if i[1] in adj_dic[i[0]]:
            continue
        else:
            adj_dic[i[0]].append(i[1])  # if True check if second node is in dictionary, if not append
    
    if i[1] not in adj_dic: # check if first node is a key already
        
        adj_dic[i[1]] = []
        adj_dic[i[1]].append(i[0])
    
    else:
        
        if i[0] in adj_dic[i[1]]:
            continue
        else:
            adj_dic[i[1]].append(i[0])  # if True check if second node is in dictionary, if not append

In [ ]:
# we saved this dictionary to a picke file so that we don't have to run the code each time
with open('adj_dic', 'wb') as fp:
    pickle.dump(adj_dic, fp)

## 2. Find the smartest Network!

Implement an algorithm that returns the set of roads (edges) that enable the user to visit all the places. We want this set to be the ones whose sum of distances is minimum.

#### Test with distance df

In [ ]:
df = distance.copy()

In [ ]:
df = df.head(20) # we'll try it with 10000 nodes

In [ ]:
df

In [ ]:
visit = [3, 1048579, 4]

#### Try with df

In [ ]:
for index, element in df.iterrows():
    for i
    print(element[0], element[1], element[2])

In [ ]:
todos = df.values

In [ ]:
todos # array with each row as a separate list

In [ ]:
walk = []
c = 0
departure = None
destination = None
for i in visit:
    for j in todos:
        if i != visit[-1:]: # last element of the list
            if c%2 == 0: # if it's the first iteration
                if i == j[0]:
                    print('first', j)
                    walk.append(j[0])
                    c += 1
                    break
            if c%2 == 1:
                if i == j[1]:
                    print('second', j)
                    walk.append(j[1])
                    c += 1
                    break
        else:
            if i == j[1]:
                    print('third', j)
                    walk.append(j[1])
                    c += 1
                    break
            

In [ ]:
walk

In [ ]:
c

## 3. Shortest Ordered Route

We'll use Dijkstra algorithm to find the shortest paths

### Create Dijkstra

In [6]:
graph = {1:{2:10, 3:3},2 :{3:1, 4:2},3:{2:4, 4:8, 5:2},4:{5:7},5:{4:9}}

In [10]:
shortest_distance = {}
predecessor = {}
unvisited = graph.copy()
path = []
for node in unvisited:
    shortest_distance[node] = float('inf')
shortest_distance[1] = 0

In [11]:
while unvisited:
    
    minNode = None
    for node in unvisited:
        if minNode is None: # for the first case, when it's empty
            minNode = node
        elif shortest_distance[node] < shortest_distance[minNode]:
            minNode = node
            
    # main part of the algo
    for childNode, weight in graph[minNode].items():
        if weight + shortest_distance[minNode] < shortest_distance[childNode]:
            shortest_distance[childNode] = weight + shortest_distance[minNode]
            predecessor[childNode] = minNode
    unvisited.pop(minNode)

In [12]:
shortest_distance

{1: 0, 2: 7, 3: 3, 4: 9, 5: 5}

In [13]:
predecessor

{2: 3, 3: 1, 4: 2, 5: 3}

### Create path

In [35]:
def find_path(graph, start, end, path=[]):
        path = path + [start]
        print("st:%s, end:%s, path:%s" % (start,end,path))
        if (start == end) or (end in graph[start]) :
            return path
        if start not in graph:
            return None
        print("available routes: %s" % graph[start])
        for node in graph[start]:
            if node not in path:
                print("next node: %s" % node)
                newpath = find_path(graph, node, end, path)
                if newpath: return newpath
        return None

In [53]:
visit = [2, 5, 4]
visit = visit[::-1] # we reverse the list of nodes we want to visit

In [54]:
megapath = [] # entire path that we want

In [55]:
while len(visit) != 1:
    path = [] # for each iteration
    start = visit[1] # second element of the list
    goal = visit[0] # first element of the lsit
    currentNode = goal # we go backwards, we want to find the predecessor
    while currentNode != start:
        
        try: # if we can find the predecessor this is what we do
            path.insert(0, currentNode)
            currentNode = predecessor[currentNode]
            print('first')
        
        
        except KeyError: # if there is not any predecessor
            
            if goal in graph[start]: # if goal is a neighbour of start 
                path = [] # path is empty
                path.insert(0, goal) # we use this as path
                dis += graph[start][goal] # and update distance
                print('here')
            
            else : 
                path = []
                path = find_path(graph, start, goal, path)
                path.insert(len(path),goal) # add the end to the list
                print('Path not reachable')
            break
   
    megapath.insert(0, path)
    visit.pop(0)

first
first
first
here
first
first
st:2, end:5, path:[2]
available routes: {3: 1, 4: 2}
next node: 3
st:3, end:5, path:[2, 3]
Path not reachable


In [56]:
merged = list(itertools.chain.from_iterable(megapath))

In [60]:
if merged[0] != start:
    merged.insert(0,start)   

In [61]:
merged

[2, 3, 5, 4]

### Test with 2 nodes

In [470]:
total_dis = 0

In [18]:
path = []
dis = 0
start = 2
goal = 5

In [20]:
currentNode = goal
while currentNode != start:
    try:
        path.insert(0, currentNode)
        currentNode = predecessor[currentNode]
        dis = shortest_distance[goal] 
        print('first')
    except KeyError:
        
       
        if goal in graph[start]:
            path = []
            dis = 0
            path.insert(0, goal)
            dis = graph[start][goal]
            print('here')
        else : 
            path = []
            path = find_path(graph, start, goal, path)
            path.insert(len(path),goal) # add the end to the list
            print('Path not reachable')
        break
path.insert(0,start)   

first
first
st:2, end:5, path:[2]
available routes: {3: 1, 4: 2}
next node: 3
st:3, end:5, path:[2, 3]
Path not reachable


In [520]:
predecessor

{2: 3, 3: 1, 4: 2, 5: 3}

In [517]:
shortest_distance

{1: 0, 2: 7, 3: 3, 4: 9, 5: 5}

In [21]:
path

[2, 2, 3, 5]

In [471]:
total_dis += dis

In [472]:
total_dis

1

In [484]:
if shortest_distance[goal] != float('inf'):
    print('shortest distance is', str(shortest_distance[goal]))
    print('and the path is'+ str(path))

shortest distance is 5
and the path is[1, 3, 5]


### Step-by Step

In [461]:
path  = []

In [462]:
visit

[5, 3, 2]

In [463]:
start = visit[1]
goal = visit[0]

In [464]:
start, goal

(3, 5)

In [446]:
currentNode = goal
while currentNode != start:
    try:
        path.insert(0, currentNode)
        currentNode = predecessor[currentNode]
        print('first')
    except KeyError:
        
        path = []
        if goal in graph[start]:
            path.insert(0, goal)
            dis += graph[start][goal]
            print('here')
        else : 
            path = []
            print('Path not reachable')
        break
megapath.insert(0, path)
visit.pop(0)

first
here


In [ ]:
if shortest_distance[goal] != float('inf'):
    print('shortest distance is', shortest_distance[goal]+dis)
    print('and the path is'+ str(path))

In [448]:
# when iteration is finished
megapath.insert(0,start)   

In [449]:
path 

[3]

In [450]:
megapath

[2, [3], [5]]

In [577]:
def find_path(graph, start, end, path=[]):
        path = path + [start]
        print("st:%s, end:%s, path:%s" % (start,end,path))
        if (start == end) or (end in graph[start]) :
            return path
        if start not in graph:
            return None
        print("available routes: %s" % graph[start])
        for node in graph[start]:
            if node not in path:
                print("next node: %s" % node)
                newpath = find_path(graph, node, end, path)
                if newpath: return newpath
        return None

In [578]:
p = []
test = find_path(graph, 1, 4, p)
test.insert(len(test),4) # add the end to the list

st:1, end:4, path:[1]
available routes: {2: 10, 3: 3}
next node: 2
st:2, end:4, path:[1, 2]


In [579]:
test

[1, 2, 4]